In [1]:
# install.packages("tidyverse")
Sys.setenv(LANGUAGE = "en")

### Data processing of LEGO product datasets

In [2]:
# Load libraries
library(tidyverse)
library(reshape2)
library(lubridate)
library(patchwork)
options(repr.plot.width = 10, repr.plot.height = 8)
Sys.setenv(LANGUAGE = "en")

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"package 'reshape2' was built under R version 4.3.2"

Attaching package: 'reshape2'


The following object is masked from 'package:tidyr':

    smiths


Warning message:
"package 'patchwork' was built under R version 4.3.2"


In [3]:
# Read color dataset
colors_data <-
  "colors.csv" %>%
   read_csv() %>%
   glimpse()

Rows: 263 Columns: 4
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): name, rgb
dbl (1): id
lgl (1): is_trans

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 263
Columns: 4
$ id       <dbl> -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17,…
$ name     <chr> "[Unknown]", "Black", "Blue", "Green", "Dark Turquoise", "Red…
$ rgb      <chr> "0033B2", "05131D", "0055BF", "237841", "008F9B", "C91A09", "…
$ is_trans <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE…


In [4]:
# Check missing values and any duplicates
colors_data %>%
  is.na() %>%
  colSums()

colors_data %>%
  duplicated() %>%
  any()

id     name      rgb is_trans 
       0        0        0        0

[1] FALSE

In [5]:
# Rename variables for clarity and easier understanding
# Remove variable we do not need
colors_data <-
  colors_data %>%
  rename(color_name = name, color_id = id) %>%
  select(-is_trans)

In [6]:
# Check missing values and any duplicates again
colors_data %>%
  is.na() %>%
  colSums()

colors_data %>%
  duplicated() %>%
  any()

color_id color_name        rgb 
         0          0          0

[1] FALSE

In [7]:
# Check unique colors
colors_data$color_id %>%
   unique() %>%
   length()

[1] 263

In [8]:
# Read sets dataset
sets_data <-
  "sets.csv" %>%
   read_csv() %>%
   glimpse()

Rows: 21993 Columns: 6
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (3): set_num, name, img_url
dbl (3): year, theme_id, num_parts

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 21,993
Columns: 6
$ set_num   <chr> "001-1", "0011-2", "0011-3", "0012-1", "0013-1", "0014-1", "…
$ name      <chr> "Gears", "Town Mini-Figures", "Castle 2 for 1 Bonus Offer", …
$ year      <dbl> 1965, 1979, 1987, 1979, 1979, 1979, 1979, 1979, 1965, 2013, …
$ theme_id  <dbl> 1, 67, 199, 143, 143, 143, 143, 189, 1, 497, 366, 366, 366, …
$ num_parts <dbl> 43, 12, 0, 12, 12, 2, 18, 15, 3, 4, 403, 35, 0, 0, 57, 18, 7…
$ img_url   <chr> "https://cdn.rebrickable.com/media/sets/001-1.jpg", "https:/…


In [9]:
# Check missing values and duplicates
sets_data %>%
  is.na() %>%
  colSums()

sets_data %>%
   duplicated() %>%
   any()

set_num      name      year  theme_id num_parts   img_url 
        0         0         0         0         0         0

[1] FALSE

In [10]:
# Rename variables for clarity and easier understanding
# Remove variable we do not need
sets_data <-
  sets_data %>%
  rename(set_id = set_num,
         set_name = name, 
         parts_count = num_parts) %>%
  select(-img_url) 

In [11]:
# Check missing values and any duplicates again
sets_data %>%
  is.na() %>%
  colSums()

sets_data %>%
  duplicated() %>%
  any()

set_id    set_name        year    theme_id parts_count 
          0           0           0           0           0

[1] FALSE

In [12]:
# Check unique years
sets_data$year %>% 
  unique() %>%
  sort()

[1] 1949 1950 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965
[16] 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980
[31] 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995
[46] 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010
[61] 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024

In [13]:
# Check the number of unique set_id
sets_data$set_id %>%
   unique() %>%
   length()

# Select the year of interests
sets_data <-
  sets_data %>%
  filter(year != 1949, year != 1950, year!= 2023, year != 2024 )

[1] 21993

In [14]:
# Read theme dataset
themes_data <-
  "themes.csv" %>%
   read_csv() %>%
   glimpse()

Rows: 468 Columns: 3
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (1): name
dbl (2): id, parent_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 468
Columns: 3
$ id        <dbl> 1, 3, 4, 16, 17, 18, 19, 20, 21, 22, 23, 34, 35, 50, 51, 52,…
$ name      <chr> "Technic", "Competition", "Expert Builder", "RoboRiders", "S…
$ parent_id <dbl> NA, 1, 1, 1, 1, 1, 1, 1, 1, NA, 22, NA, 34, NA, 50, NA, 52, …


In [15]:
# Check missing values and duplicates
themes_data %>%
  is.na() %>%
  colSums()

themes_data %>%
   duplicated() %>%
   any()

id      name parent_id 
        0         0       145

[1] FALSE

In [16]:
# Rename variables for easier understanding and select variables needed
# Remove variable we do not need
themes_data <-
  themes_data %>%
  rename(theme_name = name, theme_id = id)

In [17]:
# Check missing values and duplicates again
themes_data %>%
  is.na() %>%
  colSums()

themes_data %>%
   duplicated() %>%
   any()

theme_id theme_name  parent_id 
         0          0        145

[1] FALSE

In [18]:
# Check the number of unique theme_id
themes_data$theme_id %>%
  unique() %>%
  length()

[1] 468

In [19]:
# Read inventory parts dataset
inventory_parts_data <-
  "inventory_parts.csv" %>%
   read_csv() %>%
   glimpse()

Rows: 1183968 Columns: 6
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): part_num, img_url
dbl (3): inventory_id, color_id, quantity
lgl (1): is_spare

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 1,183,968
Columns: 6
$ inventory_id <dbl> 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, …
$ part_num     <chr> "48379c04", "48395", "stickerupn0077", "upn0342", "upn035…
$ color_id     <dbl> 72, 7, 9999, 0, 25, 47, 29, 2, 15, 15, 15, 29, 15, 15, 29…
$ quantity     <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 1, 1, 5, 2, 3, 1, …
$ is_spare     <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, F…
$ img_url      <chr> "https://cdn.rebrickable.com/media/parts/photos/1/48379c0…


In [20]:
# Check missing values and duplicates
inventory_parts_data %>%
  is.na() %>%
  colSums()

inventory_parts_data %>%
  duplicated() %>%
  any()

inventory_id     part_num     color_id     quantity     is_spare      img_url 
           0            0            0            0            0         8250

[1] FALSE

In [21]:
# Rename variables for clarity and select required variables
inventory_parts_data <-
  inventory_parts_data %>%
  rename(parts_id = part_num) %>%
  glimpse()

Rows: 1,183,968
Columns: 6
$ inventory_id <dbl> 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, …
$ parts_id     <chr> "48379c04", "48395", "stickerupn0077", "upn0342", "upn035…
$ color_id     <dbl> 72, 7, 9999, 0, 25, 47, 29, 2, 15, 15, 15, 29, 15, 15, 29…
$ quantity     <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 1, 1, 5, 2, 3, 1, …
$ is_spare     <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, F…
$ img_url      <chr> "https://cdn.rebrickable.com/media/parts/photos/1/48379c0…


In [22]:
# Remove variables we do not need
inventory_parts_data <-
  inventory_parts_data %>%
  select(-img_url)

In [23]:
# Check missing values and duplicates again
inventory_parts_data %>%
  is.na() %>%
  colSums()

inventory_parts_data %>%
  duplicated() %>%
  any()

inventory_id     parts_id     color_id     quantity     is_spare 
           0            0            0            0            0

[1] FALSE

In [24]:
# Read inventory data
inventories_data <-
  "inventories.csv" %>%
   read_csv() %>%
   glimpse()

Rows: 37422 Columns: 3
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (1): set_num
dbl (2): id, version

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 37,422
Columns: 3
$ id      <dbl> 1, 3, 4, 15, 16, 17, 19, 21, 22, 25, 26, 27, 28, 30, 31, 33, 3…
$ version <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ set_num <chr> "7922-1", "3931-1", "6942-1", "5158-1", "903-1", "850950-1", "…


In [25]:
# Check missing values and duplicates
inventories_data %>%
  is.na() %>%
  colSums()

inventories_data %>%
  duplicated() %>%
  any()

id version set_num 
      0       0       0

[1] FALSE

In [26]:
# Rename variables for clarity and easier understanding
inventories_data <-
  inventories_data %>%
  rename(inventory_id = id,
         set_id = set_num)

In [27]:
# Check missing values and duplicates again
inventories_data %>%
  is.na() %>%
  colSums()

inventories_data %>%
  duplicated() %>%
  any()

inventory_id      version       set_id 
           0            0            0

[1] FALSE

In [28]:
# Left join datasets to obtain color information over years
product_data <- 
  themes_data %>%
  left_join(sets_data, by = "theme_id") %>%
  glimpse()

Rows: 20,981
Columns: 7
$ theme_id    <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ theme_name  <chr> "Technic", "Technic", "Technic", "Technic", "Technic", "Te…
$ parent_id   <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ set_id      <chr> "001-1", "002-1", "1030-1", "1038-1", "1039-1", "1237-1", …
$ set_name    <chr> "Gears", "4.5V Samsonite Gears Motor Set", "TECHNIC I: Sim…
$ year        <dbl> 1965, 1965, 1985, 1985, 1986, 2001, 1999, 1999, 1999, 1999…
$ parts_count <dbl> 43, 3, 210, 120, 39, 56, 30, 29, 28, 26, 28, 103, 98, 64, …


In [29]:
# Check missing values and duplicates
product_data %>%
  is.na() %>%
  colSums()

product_data %>%
  duplicated() %>%
  any()

theme_id  theme_name   parent_id      set_id    set_name        year 
          0           0        9553          14          14          14 
parts_count 
         14

[1] FALSE

In [30]:
# Fill the missing value of parent-id with 0, which means no parent theme
product_data$parent_id[is.na(product_data$parent_id)] <- 0

In [31]:
# Remove missing values
product_data <- 
  product_data %>%
  na.omit()

In [32]:
# Check missing values and duplicates
product_data %>%
  is.na() %>%
  colSums()

product_data %>%
  duplicated() %>%
  any()

theme_id  theme_name   parent_id      set_id    set_name        year 
          0           0           0           0           0           0 
parts_count 
          0

[1] FALSE

In [33]:
# To have a look at the dataset
product_data %>% glimpse()

Rows: 20,967
Columns: 7
$ theme_id    <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ theme_name  <chr> "Technic", "Technic", "Technic", "Technic", "Technic", "Te…
$ parent_id   <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ set_id      <chr> "001-1", "002-1", "1030-1", "1038-1", "1039-1", "1237-1", …
$ set_name    <chr> "Gears", "4.5V Samsonite Gears Motor Set", "TECHNIC I: Sim…
$ year        <dbl> 1965, 1965, 1985, 1985, 1986, 2001, 1999, 1999, 1999, 1999…
$ parts_count <dbl> 43, 3, 210, 120, 39, 56, 30, 29, 28, 26, 28, 103, 98, 64, …


In [34]:
# Save the dataset
write.csv(product_data, file = "product_data.csv", row.names = FALSE)

In [35]:
# Left join datasets to obtain color information over years
color_data <- 
  sets_data %>%
  left_join(inventories_data, by = "set_id") %>%
  left_join(inventory_parts_data, by = "inventory_id") %>%
  left_join(colors_data, by = "color_id") %>%
  glimpse()

Rows: 1,039,223
Columns: 13
$ set_id       <chr> "001-1", "001-1", "001-1", "001-1", "001-1", "001-1", "00…
$ set_name     <chr> "Gears", "Gears", "Gears", "Gears", "Gears", "Gears", "Ge…
$ year         <dbl> 1965, 1965, 1965, 1965, 1965, 1965, 1965, 1965, 1965, 196…
$ theme_id     <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 67, 199, 143, 143, 14…
$ parts_count  <dbl> 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 12, 0, 12…
$ inventory_id <dbl> 24696, 24696, 24696, 24696, 24696, 24696, 24696, 24696, 2…
$ version      <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, …
$ parts_id     <chr> "132a", "3020", "3062c", "3404bc01", "36", "7039", "7049b…
$ color_id     <dbl> 7, 15, 15, 15, 7, 4, 15, 4, 15, 14, 1, 4, NA, NA, 0, 0, 0…
$ quantity     <dbl> 4, 4, 1, 4, 4, 6, 4, 4, 4, 4, 1, 3, NA, NA, 2, 2, 2, 2, 3…
$ is_spare     <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, F…
$ color_name   <chr> "Light Gray", "White", "White", "White", "Light Gray", "R…
$ rgb       

In [36]:
# Check missing values and duplicates
color_data %>%
  is.na() %>%
  colSums()

color_data %>%
  duplicated() %>%
  any()

set_id     set_name         year     theme_id  parts_count inventory_id 
           0            0            0            0            0            0 
     version     parts_id     color_id     quantity     is_spare   color_name 
           0         4641         4641         4641         4641         4641 
         rgb 
        4641

[1] FALSE

In [37]:
# Check how many different versions each inventory_id corresponds to
inventory_version_counts <- 
  color_data %>%
  group_by(inventory_id) %>%
  summarise(number_of_versions = n_distinct(version))

any_multiple_versions <- 
  any(inventory_version_counts$number_of_versions > 1)

# print the result
if(any_multiple_versions) {
  print("There are multiple versions corresponding to inventory_id")
} else {
  print("Each inventory_id corresponds to only one version")
}

[1] "Each inventory_id corresponds to only one version"


In [38]:
# Check number of unique colors
n_distinct(color_data$rgb)

[1] 217

In [39]:
# Remove missing values
color_data <-
  color_data %>%
  na.omit()

In [40]:
# Check number of unique colors again
n_distinct(color_data$color_id)

[1] 246

In [41]:
# Check missing values and duplicates again
color_data %>%
  is.na() %>%
  colSums()

color_data %>%
  duplicated() %>%
  any()

set_id     set_name         year     theme_id  parts_count inventory_id 
           0            0            0            0            0            0 
     version     parts_id     color_id     quantity     is_spare   color_name 
           0            0            0            0            0            0 
         rgb 
           0

[1] FALSE

In [42]:
# Calculate the total quantity for each year and RGB value
overview_color_data <-
  color_data %>%
  group_by(year, rgb) %>%
  summarise(parts_count = sum(quantity), 
            color_id = first(color_id), 
            color_name = first(color_name))


overview_color_data

`summarise()` has grouped output by 'year'. You can override using the `.groups` argument.


year,rgb,parts_count,color_id,color_name
<dbl>,<chr>,<dbl>,<dbl>,<chr>
1953,0055BF,2,1,Blue
1953,237841,26,2,Green
1953,C91A09,26,4,Red
1953,F2CD37,26,14,Yellow
1953,FFFFFF,26,15,White
1954,0055BF,18,1,Blue
1954,237841,6,2,Green
1954,9BA19D,1,7,Light Gray
1954,C91A09,70,4,Red


In [43]:
# Introduce variables for further analysis
overview_color_data <- 
  overview_color_data %>%
  mutate(color_name = case_when(
    color_name %in% c("[No Color/Any Color]") ~ "Black",  # Change [No Color/Any Color] to black
    color_name == "[Unknown]" ~ "other",  # Change [Unknown] to other
    TRUE ~ color_name  # Keep other values as they are
  ))

In [44]:
# Check the result
overview_color_data %>% 
  glimpse()

Rows: 2,381
Columns: 5
Groups: year [70]
$ year        <dbl> 1953, 1953, 1953, 1953, 1953, 1954, 1954, 1954, 1954, 1954…
$ rgb         <chr> "0055BF", "237841", "C91A09", "F2CD37", "FFFFFF", "0055BF"…
$ parts_count <dbl> 2, 26, 26, 26, 26, 18, 6, 1, 70, 18, 2, 63, 193, 1, 6, 298…
$ color_id    <dbl> 1, 2, 4, 14, 15, 1, 2, 7, 4, 14, 47, 15, 1, 9999, 2, 4, 14…
$ color_name  <chr> "Blue", "Green", "Red", "Yellow", "White", "Blue", "Green"…


In [45]:
# Save the dataset
write.csv(overview_color_data, file = "colour_data.csv", row.names = FALSE)

### Data processing of LEGO market dataset

In [46]:
# Read LEGO market data: rating data
rating_data <-
  "kaggle_sets.csv" %>%
  read_csv()

Rows: 14936 Columns: 17
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (8): Set_ID, Name, Theme, Theme_Group, Subtheme, Category, Packaging, Av...
dbl (9): Year, Num_Instructions, Pieces, Minifigures, Owned, Rating, USD_MSR...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [47]:
# Rename variables for clarity and select variables of interests
rating_data <-
  rating_data %>%
    rename(set_id = Set_ID,
           set_name = Name,
           year = Year,
           theme_name = Theme,
           theme_group = Theme_Group,
           parts_count = Pieces,
           minifigures_count = Minifigures,
           rating = Rating,
           retail_price = USD_MSRP,
           resale_price = Current_Price) %>%
  select(year, set_id, set_name, theme_name, theme_group, 
         parts_count, minifigures_count, rating, 
         retail_price, resale_price)

In [48]:
# Have a look at the data
rating_data %>% 
  glimpse()

Rows: 14,936
Columns: 10
$ year              <dbl> 1975, 1975, 1975, 1975, 1975, 1975, 1975, 1975, 1975…
$ set_id            <chr> "75-1", "77-1", "077-1", "78-1", "78-3", "133-1", "1…
$ set_name          <chr> "PreSchool Set", "PreSchool Set", "Pre-School Set", …
$ theme_name        <chr> "PreSchool", "PreSchool", "Duplo", "PreSchool", "Sam…
$ theme_group       <chr> "Pre-school", "Pre-school", "Pre-school", "Pre-schoo…
$ parts_count       <dbl> 16, 20, 21, 32, 330, 87, 86, 83, 82, 293, 369, 526, …
$ minifigures_count <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, 5, NA, NA, NA, N…
$ rating            <dbl> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ retail_price      <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ resale_price      <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …


In [49]:
# Read LEGO market data: price data
price_data <-
  "brickset_sets.csv" %>%
  read_csv()

Rows: 19195 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (6): Number, Theme, Subtheme, Set name, Launch date, Exit date
dbl (8): Year, Pieces, RRP (GBP), RRP (USD), RRP (CAD), RRP (EUR), Value new...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [50]:
# Have a glimpse at the dataset
price_data %>% 
  glimpse()
# Check the statistics of price data
price_data %>%
  summary()
# Check missing values
price_data %>%
  is.na() %>%
  colSums()
# Check any duplicates in the dataset
price_data %>%
  duplicated() %>%
  any()

Rows: 19,195
Columns: 14
$ Number             <chr> "700_1_4-1", "700_3_4-1", "700_B-1", "700_F-1", "70…
$ Theme              <chr> "System", "System", "System", "System", "System", "…
$ Subtheme           <chr> "Supplemental", "Supplemental", "Supplemental", "Su…
$ Year               <dbl> 1953, 1953, 1953, 1953, 1954, 1954, 1954, 1954, 195…
$ `Set name`         <chr> "Individual 1 x 2 Bricks", "Individual 2 x 3 Bricks…
$ Pieces             <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ `RRP (GBP)`        <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ `RRP (USD)`        <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ `RRP (CAD)`        <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ `RRP (EUR)`        <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ `Value new (USD)`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ `Value used (USD)` <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ `Launch date`

    Number             Theme             Subtheme              Year     
 Length:19195       Length:19195       Length:19195       Min.   :1953  
 Class :character   Class :character   Class :character   1st Qu.:2000  
 Mode  :character   Mode  :character   Mode  :character   Median :2011  
                                                          Mean   :2006  
                                                          3rd Qu.:2017  
                                                          Max.   :2022  
                                                                        
   Set name             Pieces          RRP (GBP)        RRP (USD)     
 Length:19195       Min.   :    0.0   Min.   :  0.00   Min.   :  0.00  
 Class :character   1st Qu.:   22.0   1st Qu.:  5.99   1st Qu.:  6.00  
 Mode  :character   Median :   67.0   Median : 14.99   Median : 15.00  
                    Mean   :  220.9   Mean   : 27.44   Mean   : 30.61  
                    3rd Qu.:  235.0   3rd Qu.: 29.99   3

Number            Theme         Subtheme             Year 
               0                0             3548                0 
        Set name           Pieces        RRP (GBP)        RRP (USD) 
               0             4059            10302             7871 
       RRP (CAD)        RRP (EUR)  Value new (USD) Value used (USD) 
           13190            14712             6612             7736 
     Launch date        Exit date 
           12193            12193

[1] FALSE

In [51]:
# Rename variables for clarity and select variables of interests
price_data <- 
  price_data %>% 
  rename(set_id = Number,
         theme_name = Theme,
         year = Year,
         set_name = `Set name`,
         parts_count = Pieces,
         retail_price = `RRP (USD)`,
         resale_price_used = `Value used (USD)`,
         resale_price_new = `Value new (USD)`,
         launch_date = `Launch date`,
         exit_date = `Exit date`) %>%
  select(year, set_id, set_name, theme_name, parts_count,
         retail_price, resale_price_used, resale_price_new,
         launch_date, exit_date)

In [52]:
# have a look at the dataset
price_data %>%
  glimpse()

Rows: 19,195
Columns: 10
$ year              <dbl> 1953, 1953, 1953, 1953, 1954, 1954, 1954, 1954, 1954…
$ set_id            <chr> "700_1_4-1", "700_3_4-1", "700_B-1", "700_F-1", "700…
$ set_name          <chr> "Individual 1 x 2 Bricks", "Individual 2 x 3 Bricks"…
$ theme_name        <chr> "System", "System", "System", "System", "System", "S…
$ parts_count       <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ retail_price      <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ resale_price_used <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ resale_price_new  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ launch_date       <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ exit_date         <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …


In [53]:
# Create a join data that preparing for joining, selecting variables of interests
join_data <-
  rating_data %>% 
  select(set_id, rating, minifigures_count, rating)

In [54]:
# Have a look at the dataset
join_data

set_id,rating,minifigures_count
<chr>,<dbl>,<dbl>
75-1,0.0,NA
77-1,0.0,NA
077-1,0.0,NA
78-1,0.0,NA
78-3,0.0,NA
133-1,0.0,NA
134-1,0.0,NA
136-1,0.0,NA
137-2,0.0,NA


In [55]:
# Join price dataset and join dataset
total_data <- 
  left_join(price_data, join_data, by = "set_id") %>%
  glimpse()

# Check missing values
total_data %>%
  is.na() %>%
  colSums()

# Check any duplicates in the dataset
total_data %>%
  duplicated() %>%
  any()

Rows: 19,195
Columns: 12
$ year              <dbl> 1953, 1953, 1953, 1953, 1954, 1954, 1954, 1954, 1954…
$ set_id            <chr> "700_1_4-1", "700_3_4-1", "700_B-1", "700_F-1", "700…
$ set_name          <chr> "Individual 1 x 2 Bricks", "Individual 2 x 3 Bricks"…
$ theme_name        <chr> "System", "System", "System", "System", "System", "S…
$ parts_count       <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ retail_price      <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ resale_price_used <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ resale_price_new  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ launch_date       <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ exit_date         <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ rating            <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ minifigures_count <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …


year            set_id          set_name        theme_name 
                0                 0                 0                 0 
      parts_count      retail_price resale_price_used  resale_price_new 
             4059              7871              7736              6612 
      launch_date         exit_date            rating minifigures_count 
            12193             12193              4773             11690

[1] FALSE

In [56]:
# Transfer these variables to date format
total_data$launch_date <- dmy(total_data$launch_date)
total_data$exit_date <- dmy(total_data$exit_date)

# Filter data with date ealier than or equal to "31/12/2022"
total_data <- 
  total_data %>% 
  filter(exit_date <= dmy("31/12/2022"),
         )

# Check the result
total_data

year,set_id,set_name,theme_name,parts_count,retail_price,resale_price_used,resale_price_new,launch_date,exit_date,rating,minifigures_count
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<date>,<dbl>,<dbl>
1996,1796-1,Freestyle Bucket,Freestyle,641,20.00,30.00,NA,1996-06-01,2000-09-30,0.0,NA
1996,1815-1,Paradisa Lifeguard,Town,36,NA,41.24,107.70,1996-01-01,1997-12-31,0.0,2
1996,1817-1,Sea Plane with Hut and Boat,Town,135,15.00,32.74,29.75,1996-03-01,2001-12-31,0.0,3
1996,4559-1,Cargo Railway,Trains,842,130.00,216.43,762.15,1996-07-01,1999-01-30,4.3,7
1996,4565-1,Freight and Crane Railway,Trains,914,140.00,243.55,589.41,1996-09-01,2002-12-31,4.4,3
1996,5820-1,Belville Garden Fun,Belville,29,NA,9.05,32.53,1996-01-01,1998-12-31,0.0,1
1996,5835-1,Belville Dance Studio,Belville,36,NA,21.54,15.00,1996-08-01,1998-12-31,0.0,1
1996,5875-1,Belville Hospital Ward,Belville,96,NA,43.78,120.00,1996-01-01,2001-12-31,0.0,4
1996,5876-1,Belville Hospital Ward,Belville,96,30.00,40.88,75.00,1996-08-01,1998-12-31,0.0,4


In [57]:
# Filter out the retail price, and resale price with 0, which are regarded as missing values
total_data <-
  total_data %>%
  filter(retail_price != 0,
         resale_price_used != 0,
         resale_price_new != 0,
         resale_price_new >= resale_price_used) %>%
  mutate(used_set_profit = round(resale_price_used - retail_price, 2),
         new_set_profit = round(resale_price_new - retail_price, 2))

In [58]:
# Check the result
total_data

year,set_id,set_name,theme_name,parts_count,retail_price,resale_price_used,resale_price_new,launch_date,exit_date,rating,minifigures_count,used_set_profit,new_set_profit
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>
1996,4559-1,Cargo Railway,Trains,842,130.00,216.43,762.15,1996-07-01,1999-01-30,4.3,7,86.43,632.15
1996,4565-1,Freight and Crane Railway,Trains,914,140.00,243.55,589.41,1996-09-01,2002-12-31,4.4,3,103.55,449.41
1996,5876-1,Belville Hospital Ward,Belville,96,30.00,40.88,75.00,1996-08-01,1998-12-31,0.0,4,10.88,45.00
1996,6024-1,Bandit Ambush,Castle,59,6.50,78.79,213.75,1996-06-01,1998-12-31,3.9,2,72.29,207.25
1996,6135-1,Spy Shark,Aquazone,57,8.75,11.79,20.00,1996-01-01,1997-12-31,3.6,1,3.04,11.25
1996,6145-1,Crystal Crawler,Aquazone,96,12.00,17.05,94.67,1996-04-01,1998-12-31,3.8,1,5.05,82.67
1996,6190-1,Shark's Crystal Cave,Aquazone,258,50.00,106.77,400.00,1996-07-01,1997-12-31,4.4,2,56.77,350.00
1996,6244-1,Armada Sentry,Pirates,71,8.75,45.89,166.40,1996-04-01,1996-12-31,3.5,1,37.14,157.65
1996,6334-1,Wave Jump Racers,Town,181,20.00,32.60,61.48,1996-07-01,1997-12-31,4.0,4,12.60,41.48


In [59]:
# Modify the theme name to right format
total_data <- 
  total_data %>%
  mutate(theme_name = 
         ifelse(theme_name == "Avatar The Last Airbender", 
                             "Avatar: The Last Airbender", 
                             theme_name))

In [60]:
# Create the market data for the analysis of LEGO market
# Select variables of interests
market_data <-
  total_data %>%
  select(year, set_id, theme_name, set_name,
         retail_price, rating, 
         used_set_profit, new_set_profit)

In [61]:
# Check the missing values
market_data %>%
  is.na() %>%
  colSums()

# Check any duplicates
market_data %>%
  duplicated() %>%
  any()

year          set_id      theme_name        set_name    retail_price 
              0               0               0               0               0 
         rating used_set_profit  new_set_profit 
            209               0               0

[1] FALSE

In [62]:
# Remove the missing values
market_data <- 
  market_data %>%
  na.omit()

In [63]:
# Check the statistics of the data
market_data %>%
  glimpse()

Rows: 5,002
Columns: 8
$ year            <dbl> 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, …
$ set_id          <chr> "4559-1", "4565-1", "5876-1", "6024-1", "6135-1", "614…
$ theme_name      <chr> "Trains", "Trains", "Belville", "Castle", "Aquazone", …
$ set_name        <chr> "Cargo Railway", "Freight and Crane Railway", "Belvill…
$ retail_price    <dbl> 130.00, 140.00, 30.00, 6.50, 8.75, 12.00, 50.00, 8.75,…
$ rating          <dbl> 4.3, 4.4, 0.0, 3.9, 3.6, 3.8, 4.4, 3.5, 4.0, 3.9, 3.3,…
$ used_set_profit <dbl> 86.43, 103.55, 10.88, 72.29, 3.04, 5.05, 56.77, 37.14,…
$ new_set_profit  <dbl> 632.15, 449.41, 45.00, 207.25, 11.25, 82.67, 350.00, 1…


In [64]:
# Save the dataset
write.csv(market_data, file = "market_data.csv", row.names = FALSE)

In [65]:
# Create the dataset for price prediction model
# Select variables of interest
model_data <-
  total_data %>%
  select(set_id, set_name, theme_name,
         parts_count, rating, retail_price, 
         minifigures_count, resale_price_used, resale_price_new,
         launch_date, exit_date)

In [66]:
# Check the dataset
model_data

set_id,set_name,theme_name,parts_count,rating,retail_price,minifigures_count,resale_price_used,resale_price_new,launch_date,exit_date
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<date>
4559-1,Cargo Railway,Trains,842,4.3,130.00,7,216.43,762.15,1996-07-01,1999-01-30
4565-1,Freight and Crane Railway,Trains,914,4.4,140.00,3,243.55,589.41,1996-09-01,2002-12-31
5876-1,Belville Hospital Ward,Belville,96,0.0,30.00,4,40.88,75.00,1996-08-01,1998-12-31
6024-1,Bandit Ambush,Castle,59,3.9,6.50,2,78.79,213.75,1996-06-01,1998-12-31
6135-1,Spy Shark,Aquazone,57,3.6,8.75,1,11.79,20.00,1996-01-01,1997-12-31
6145-1,Crystal Crawler,Aquazone,96,3.8,12.00,1,17.05,94.67,1996-04-01,1998-12-31
6190-1,Shark's Crystal Cave,Aquazone,258,4.4,50.00,2,106.77,400.00,1996-07-01,1997-12-31
6244-1,Armada Sentry,Pirates,71,3.5,8.75,1,45.89,166.40,1996-04-01,1996-12-31
6334-1,Wave Jump Racers,Town,181,4.0,20.00,4,32.60,61.48,1996-07-01,1997-12-31


In [67]:
# Check missing values
model_data %>%
  is.na() %>%
  colSums()

# Check any duplicates in the dataset
total_data %>%
  duplicated() %>%
  any()

set_id          set_name        theme_name       parts_count 
                0                 0                 0                37 
           rating      retail_price minifigures_count resale_price_used 
              209                 0              1521                 0 
 resale_price_new       launch_date         exit_date 
                0                 0                 0

[1] FALSE

In [68]:
# To introduce two variables for price prediction
model_data <- 
  model_data %>%
  mutate(
    lifecycle_duration =  as.numeric(exit_date - launch_date), 
    days_since_launch =  as.numeric(dmy("31/12/2022") - launch_date)
        )

In [69]:
# Select the variables of interests
model_data <-
  model_data %>%
  select(theme_name,
         parts_count, minifigures_count, rating, 
         retail_price, lifecycle_duration, days_since_launch,
         resale_price_used, resale_price_new)

In [70]:
# Check missing values
model_data %>%
  is.na() %>%
  colSums()

# Check any duplicates in the dataset
model_data %>%
  duplicated() %>%
  any()

theme_name        parts_count  minifigures_count             rating 
                 0                 37               1521                209 
      retail_price lifecycle_duration  days_since_launch  resale_price_used 
                 0                  0                  0                  0 
  resale_price_new 
                 0

[1] FALSE

In [74]:
# Remove missing values
model_data <-
  model_data %>%
  na.omit()

# Check the dataset
model_data %>%
  glimpse()

Rows: 3,653
Columns: 9
$ theme_name         <chr> "Trains", "Trains", "Belville", "Castle", "Aquazone…
$ parts_count        <dbl> 842, 914, 96, 59, 57, 96, 258, 71, 181, 329, 58, 15…
$ minifigures_count  <dbl> 7, 3, 4, 2, 1, 1, 2, 1, 4, 6, 1, 2, 2, 1, 2, 8, 1, …
$ rating             <dbl> 4.3, 4.4, 0.0, 3.9, 3.6, 3.8, 4.4, 3.5, 4.0, 3.9, 3…
$ retail_price       <dbl> 130.00, 140.00, 30.00, 6.50, 8.75, 12.00, 50.00, 8.…
$ lifecycle_duration <dbl> 943, 2312, 882, 943, 730, 1004, 548, 274, 548, 548,…
$ days_since_launch  <dbl> 9679, 9617, 9648, 9709, 9861, 9770, 9679, 9770, 967…
$ resale_price_used  <dbl> 216.43, 243.55, 40.88, 78.79, 11.79, 17.05, 106.77,…
$ resale_price_new   <dbl> 762.15, 589.41, 75.00, 213.75, 20.00, 94.67, 400.00…


In [75]:
# Save the dataset
write.csv(model_data, file = "model_data.csv", row.names = FALSE)